In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv('trainset.csv')
test = pd.read_csv('test_x.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [4]:
train.fillna(method='bfill', inplace=True)

/var/folders/8s/4g_l84wj1tjg71_xcwhr91yw0000gn/T/ipykernel_32453/3950879406.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train.fillna(method='bfill', inplace=True)


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
categorical_feature = train.dtypes[train.dtypes=='object'].index.values
for feature in categorical_feature:
    train[feature] = le.fit_transform(train[feature])
c_f = test.dtypes[test.dtypes=='object'].index.values
for f in c_f:
    test[f] = le.fit_transform(test[f])

In [6]:
for col in train.columns:
    q1 = train[col].quantile(0.0001)
    q3 = train[col].quantile(0.9999)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    train_encoded = train[(train[col] >= lower_bound) & (train[col] <= upper_bound)]

In [7]:
train_x = train.drop('selling_price', axis=1)
train_y = train['selling_price']

In [8]:
train_x.isnull().sum()

id                   0
name                 0
year                 0
km_driven            0
Region               0
State or Province    0
City                 0
fuel                 0
seller_type          0
transmission         0
owner                0
mileage              0
engine               0
max_power            0
torque               0
seats                0
sold                 0
dtype: int64

In [9]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(train_x, train_y)
score = model.score(train_x, train_y)
print(score)

0.9928863581222646


In [10]:
y = model.predict(test)
sample_submission['selling_price'] = y
sample_submission.to_csv('submission.csv', index=False)
import files